#Leslie Janeth Quincosa Ramírez

Métodos numéricos

29/09/23

**Problema 1** Aplica el método de la potencia y potencia inversa para el cálculo del autovalor y autovector dominante de la matriz propuesta y su inversa.

$$
A=\left(\begin{array}{ccc}
15 & -2 & 2 \\
1 & 10 & -3 \\
-2 & 1 & 0
\end{array}\right)
$$

Da la tabla de iterados, la tolerancia usada y el vector inicial.

(a) Con los resultados obtenidos da una aproximación para el número de condición de la matriz.

(b) Implementa a alguna técnica para disminuir el número de iterados obtenidos. Muestra la nueva tabla de iterados.

In [1]:
import numpy as np

#Funciones preeliminares

In [2]:
#Matriz triangular superior

def TriangularSup(A, b):
  det = 1
  n = A.shape[0]
  x = np.zeros(n)
  tol = 0.0000001

  for i in reversed(range(n)): #Es la misma función que TriangularInf solo que empezamos de n a 0
    det *= A[i][i]

    if abs(det) < tol:
      print("No hay solución al sistema")
      return

    sum = 0
    for j in range(i, n):
      sum += x[j]*(A[i][j])

    x[i] = (b[i]- sum)/A[i][i]
  return x

In [3]:
#Metodo de eliminación Gauss-Jordan

def GaussJordan(A_original, b_original):
  A = A_original.astype('float64')
  b = b_original.astype('float64')
  n = A.shape[0]
  tol = 0.00001

  for k in range(n):
    fila_pivote = k
    for j in range(k+1, n):
      if abs(A[j, k]) > abs(A[fila_pivote, k]):
        fila_pivote = j

      #Pivoteo
      A[[k, fila_pivote]] = A[[fila_pivote, k]] #intercambio la fila donde está el pivote mayor
      b[k], b[fila_pivote] = b[fila_pivote], b[k]

    # checar si pivote es practicasmente 0
    if abs(A[k, k ]) < tol:
      break
    for i in range(k+1, n):
      m_ik = A[i, k]/A[k, k]

      for j in range(k, n):
        A[i,j] = A[i, j] - m_ik*A[k, j]

      b[i] = b[i] - m_ik*b[k]

  #print(A)
  #print(b)

  return TriangularSup(A, b)

#Método de la potencia simple

In [4]:
#Metodo de la potencia

def potencia(A, Tol, max_iter):
  lamb = 0
  k = 0
  x_0 = np.random.rand(len(A))#inicializo el vector
  x_0 = x_0/np.linalg.norm(x_0) #normalizando el vector propio
  error = np.linalg.norm(A@x_0 - lamb*x_0)/np.linalg.norm(x_0) #Error epsilon

  while k < max_iter and error > Tol:
    x_k = A@x_0 #xk en el vector propio
    x_0 = x_k/np.linalg.norm(x_k) #normalizando el vector propio
    lamb = x_0.T@A@x_0 #lambda con cociente de Rayleigh
    error = np.linalg.norm(A@x_0 - lamb*x_0)/np.linalg.norm(x_0)
    k = k + 1
    print("iteración: ", k)
    print("EigenVector v: ", x_0)
    print("Eigenvalor lambda: ", lamb)
    print("------------------------------------------------")

  return lamb, x_k/np.linalg.norm(x_k), k

#Método de la potencia con aceleración

In [5]:
#Metodo de la potencia

def metodo_potencia(A, Tol, max_iter):
  lamb_k = np.inf
  k = 0
  x_0 = np.random.rand(len(A))#inicializo el vector
  x_0 = x_0/np.linalg.norm(x_0) #normalizando el vector propio

  while k < max_iter:
    x_k = A@x_0 #xk en el vector propio
    x_0 = x_k/np.linalg.norm(x_k) #normalizando el vector propio
    lamb_k1 = x_0.T@A@x_0 #lambda con cociente de Rayleigh
    print("iteración: ", k)
    print("EigenVector v: ", x_0)
    print("Eigenvalor lambda: ", lamb_k1)
    print("------------------------------------------------")
    if abs(lamb_k - lamb_k1)< Tol:
      return lamb_k, x_k/np.linalg.norm(x_k), k

    lamb_k = lamb_k1
    k = k + 1


#Método de la potencia inversa

In [6]:
def potencia_inv(A, Tol, max_iter):
  k =0
  lamb = 0
  x_0 = np.random.rand(len(A)) #inicializo el vector
  x_0 = x_0/np.linalg.norm(x_0) #normalizo el vector
  epsilon = np.inf #error de la ec caracteristica

  while k <max_iter and epsilon > Tol:
    x_k = GaussJordan(A, x_0) #Resolver A@x = x_0  donde x_k es el vector propio con Gauss Jordan
    x_0 = x_k/np.linalg.norm(x_k)
    lamb = 1/(x_0.T@A@x_0) #lambda con cociente de Rayleigh
    epsilon = np.linalg.norm(A@x_0 - 1/lamb*x_0)
    k = k+1
    print("iteración: ", k)
    print("EigenVector v: ", x_0)
    print("Eigenvalor lambda: ", lamb)
    print("------------------------------------------------")

  return 1/lamb, x_k/np.linalg.norm(x_k), k

In [31]:
#Circulos de Gershgorin \mu es el desplazamiento a la matriz
def potencia_inv_desplazamiento(A, Tol, max_iter, mu):
  k =0
  lamb = 0
  x_0 = np.random.rand(len(A)) #inicializo el vector
  x_0 = x_0/np.linalg.norm(x_0) #normalizo el vector
  A_new = A - mu*np.identity(len(A))
  epsilon = np.inf

  while k <max_iter and epsilon > Tol:
    x_k = GaussJordan(A_new, x_0) #Resolver A@x = x_0  donde x_k es el vector propio con Gauss Jordan
    x_0 = x_k/np.linalg.norm(x_k)
    lamb = 1/(x_0.T@A_new@x_0) #lambda con cociente de Rayleigh
    epsilon = np.linalg.norm(A_new@x_0 - 1/lamb*x_0)
    k = k+1
    print("iteración: ", k)
    print("EigenVector v: ", x_0)
    print("Eigenvalor lambda: ", lamb)
    print("------------------------------------------------")

  return 1/lamb + mu, x_k/np.linalg.norm(x_k), k

#Calculos ejercicio 1

In [8]:
#Matriz del problema a

A = np.array([[15, -2, 2],[1, 10, -3],[-2, 1, 0]])

print("El máximo valor propio de A es ", np.max(np.linalg.eigvals(A)))
print("El mínimo valor propio de A es ", np.min(np.linalg.eigvals(A)))

El máximo valor propio de A es  14.102555760088634
El mínimo valor propio de A es  0.5120848255718702


Calculo del λ máximo del **método de la potencia** y su valor propio asociado

In [9]:
#Método de la potencia simple
lamb_max, x_max, k = potencia(A, 0.00001, 50)

print("lambda máxima= ", lamb_max)
print("x: ", x_max)
print("iteraciones: ", k)

iteración:  1
EigenVector v:  [ 6.61601924e-01  7.49855080e-01 -5.02803747e-04]
Eigenvalor lambda:  11.69323149927575
------------------------------------------------
iteración:  2
EigenVector v:  [ 0.7173165   0.69503461 -0.04882551]
Eigenvalor lambda:  12.118186516375468
------------------------------------------------
iteración:  3
EigenVector v:  [ 0.76324417  0.64323541 -0.06088142]
Eigenvalor lambda:  12.463019355024008
------------------------------------------------
iteración:  4
EigenVector v:  [ 0.80380345  0.59067763 -0.0707103 ]
Eigenvalor lambda:  12.78924575179211
------------------------------------------------
iteración:  5
EigenVector v:  [ 0.8377395   0.54027195 -0.07936463]
Eigenvalor lambda:  13.079199742927791
------------------------------------------------
iteración:  6
EigenVector v:  [ 0.8647758   0.49462215 -0.08667037]
Eigenvalor lambda:  13.32206926949113
------------------------------------------------
iteración:  7
EigenVector v:  [ 0.88553415  0.45525121 

In [10]:
#Método de potencia con una mejora y tabla de iteraciones
lamb_max, x_max, k= metodo_potencia(A, 0.00001, 50)

iteración:  0
EigenVector v:  [ 0.99565672 -0.02094168 -0.09071465]
Eigenvalor lambda:  14.891421296789677
------------------------------------------------
iteración:  1
EigenVector v:  [ 0.9883976   0.07070514 -0.13442833]
Eigenvalor lambda:  14.653064037282437
------------------------------------------------
iteración:  2
EigenVector v:  [ 0.98120428  0.14285031 -0.129738  ]
Eigenvalor lambda:  14.542390605629219
------------------------------------------------
iteración:  3
EigenVector v:  [ 0.97336234  0.19222368 -0.12496321]
Eigenvalor lambda:  14.44195171178223
------------------------------------------------
iteración:  4
EigenVector v:  [ 0.96645681  0.22631908 -0.12141214]
Eigenvalor lambda:  14.359012893522078
------------------------------------------------
iteración:  5
EigenVector v:  [ 0.96087307  0.2502115  -0.11881562]
Eigenvalor lambda:  14.294250323979885
------------------------------------------------
iteración:  6
EigenVector v:  [ 0.95653316  0.26715941 -0.1169194

Calculo del λ mínimo del **método de la potencia inversa** y su valor propio asociado

In [32]:
lamb_min, x_min, k= potencia_inv(A, 0.00001, 50)

print("lambda mínima= ", lamb_min)
print("x: ", x_min)
print("iteraciones: ", k)

iteración:  1
EigenVector v:  [-0.05717604  0.32107264  0.94532707]
Eigenvalor lambda:  2.0356928692631255
------------------------------------------------
iteración:  2
EigenVector v:  [-0.0869876   0.30921175  0.94700647]
Eigenvalor lambda:  1.9574519314973091
------------------------------------------------
iteración:  3
EigenVector v:  [-0.08807576  0.30875685  0.94705431]
Eigenvalor lambda:  1.9529610136016444
------------------------------------------------
iteración:  4
EigenVector v:  [-0.08811573  0.30873939  0.94705629]
Eigenvalor lambda:  1.9528066941364621
------------------------------------------------
iteración:  5
EigenVector v:  [-0.0881172   0.30873871  0.94705637]
Eigenvalor lambda:  1.9528016312281844
------------------------------------------------
lambda mínima=  0.5120847832204367
x:  [-0.0881172   0.30873871  0.94705637]
iteraciones:  5


In [39]:
#Usando Circulos de Gershgorin donde 14 es el desplazamiento
lamb_min, x_min, k = potencia_inv_desplazamiento(A, 0.00001, 50, 14)

print("lambda máxima= ", lamb_min)
print("x: ", x_max)
print("iteraciones: ", k)

iteración:  1
EigenVector v:  [ 0.93955452  0.32104334 -0.11903143]
Eigenvalor lambda:  21.313144936080356
------------------------------------------------
iteración:  2
EigenVector v:  [ 0.94370617  0.31135759 -0.11169207]
Eigenvalor lambda:  9.62670302152756
------------------------------------------------
iteración:  3
EigenVector v:  [ 0.94358895  0.31170408 -0.11171594]
Eigenvalor lambda:  9.75417956876017
------------------------------------------------
iteración:  4
EigenVector v:  [ 0.94359228  0.31169374 -0.11171669]
Eigenvalor lambda:  9.750698448336891
------------------------------------------------
lambda máxima]=  14.102556755836353
x:  [ 0.94359475  0.31168594 -0.11171763]
iteraciones:  4


(a) Con los resultados obtenidos da una aproximación para el número de condición de la matriz.

Hemos visto en clase que una cota la para el número de condición es
$$ \frac{| \lambda_{max} |}{|\lambda_{min}|} \leq k(A) \leq \frac{| \lambda_{max} |}{|\lambda_{min}|} + ɛ$$

Así que dados los valores de $\lambda_{max}$ y $\lambda_{min}$ con el método de la potencai y potencia inversa respectivamente, tenemos que

$$27.5394 \leq 33.24309  $$

Tenemos la cota inferior, más no conocemos el ɛ tal que nos dé la cota superior.

In [13]:
k_A = abs(lamb_max)/abs(lamb_min)
k_A

137.51032326996537

In [14]:
np.linalg.cond(A, 2)

33.243093514119735

#4. Usa una técnica numérica para determinar todos los valores propios y los vectores propios asociados a la matriz:

$$
A=\left(\begin{array}{lllll}
4 & 2 & 0 & 0 & 0 \\
2 & 4 & 2 & 0 & 0 \\
0 & 2 & 4 & 2 & 0 \\
0 & 0 & 2 & 4 & 2 \\
0 & 0 & 0 & 2 & 4
\end{array}\right)
$$

¿Qué técnica usaste? Da muestra de tus resultados computacionales.

In [15]:
#Matriz del ejercicio
A_4 = np.array([[4, 2, 0, 0, 0],
                [2, 4, 2, 0, 0],
                [0, 2, 4, 2, 0],
                [0, 0, 2, 4, 2],
                [0, 0, 0, 2, 4]])

#Matriz Householder

In [16]:
def HouseHolder(v):
  n = v.size
  return np.identity(n) - 2*(np.outer(v, v.T))/np.linalg.norm(v.T@v)

In [17]:
#Calculo de eigenvector dado el eigenvalor
def eigenvector(A, eigenvalor):
    # Ecuación (A - lambI)x=0
    A_minus_lambdaI = A - eigenvalor * np.identity(A.shape[0])

    #Si usamos como vector solucion al vector 0, nos arroja la sol trivial
    # Nos aproximamos mediante la uniform distribution  [0, 1)
    eigenvector = np.linalg.solve(A_minus_lambdaI, np.random.rand(A.shape[0]))

    #Normalizamos el vector
    eigenvector = eigenvector / np.linalg.norm(eigenvector)

    return eigenvector

#Método de la potencia con Householder

Se utilizó el metodo de la potencia en combinación con la matriz Hoseholder. De modo que buscamos en la matriz $A$ dada, el valor propio maximo $\lambda_1$ y su vector propio asociado $v_1$ . Después con ese vector propio $v_1$ construimos a la matriz Householder como vimos en el inciso 3. Luego creamos a la matriz

$$B_1 = H A H$$

cuya submatriz de $n-1 \times n-1$ ahora contendra a los valores propios de la matriz $A$ original.

Procedemos de forma iterativa, encontrando el valor propio de $B_1$ con metodo de la potencia, extrayendo el vector propio asociado al siguiente valor propio y contruyendo una matriz Householder, y así $B_2 = H_1 B_1 H_2$ es la siguiente matriz a la que extraemos la submatriz con los valores y vectores propios.

Finalmente obtenemos los $5$ valores propios y vectores propios de $A \in M^{6 \times 6}$.

In [18]:
def Potencia_HouseHolder(A, Tol, max_iter):
  eigenvalores = []
  eigenvectores = []
  #Calculo el primer valor propio y vector propio con metodo de la potencia
  lamb, x, k = metodo_potencia(A, Tol, max_iter)
  eigenvalores.append(lamb)
  eigenvectores.append(x)

  n = A.shape[0]
  e1 = np.eye(1, n, 0)[0]
  #Este es el vector para la factorización QR
  v = x - e1
  H = HouseHolder(v)
  A1 = H@A@H

  #Reducimos la dimensión de la matriz para encontrar una submatriz
  #Esta submatriz tiene los mismos valores propios que A
  for i in range(1, n):
    #print(i)
    Bi = A1[1:,1:]
    if i == n-1:
      lambi = Bi[0,0]
      vectori = eigenvector(A, lambi)
      eigenvalores.append(lambi)
      eigenvectores.append(vectori)
      break
    #print(Bi.shape, "B_i")
    #Volvemos a aplicar metodo de la potencia
    lambi, xi, ki = metodo_potencia(Bi, Tol, max_iter)
    vectori = eigenvector(A, lambi)
    #print(xi.size, "xi")
    #print(np.eye(1, n-i, 0)[0], "e1")
    vi = xi - np.eye(1, n-i, 0)[0]
    H = HouseHolder(vi)
    Ai = H@Bi@H
    #print(Ai)
    A1 = Ai
    eigenvalores.append(lambi)
    eigenvectores.append(vectori)

  return eigenvalores, np.array(eigenvectores)

In [19]:
np.linalg.eigvals(A_4)

array([7.46410162, 6.        , 4.        , 0.53589838, 2.        ])

In [20]:
values, vectors = Potencia_HouseHolder(A_4, 0.00001, 50)

print("Eigenvalores= ", values)
print("EigenVectores= ", vectors)


iteración:  0
EigenVector v:  [0.30620078 0.5673691  0.43433257 0.40710088 0.47953882]
Eigenvalor lambda:  7.168774354565605
------------------------------------------------
iteración:  1
EigenVector v:  [0.32523495 0.51696814 0.50810889 0.47638901 0.3766232 ]
Eigenvalor lambda:  7.409155444744342
------------------------------------------------
iteración:  2
EigenVector v:  [0.31453185 0.50308372 0.54142088 0.49506307 0.33128911]
Eigenvalor lambda:  7.450649478163027
------------------------------------------------
iteración:  3
EigenVector v:  [0.30377429 0.49963831 0.55836439 0.49983041 0.31061472]
Eigenvalor lambda:  7.460398809326531
------------------------------------------------
iteración:  4
EigenVector v:  [0.29678269 0.49895447 0.56724913 0.50089104 0.30050136]
Eigenvalor lambda:  7.463043919441541
------------------------------------------------
iteración:  5
EigenVector v:  [0.29277171 0.49895989 0.57195801 0.50099433 0.29528371]
Eigenvalor lambda:  7.463794523807345
-----

In [21]:
#Comprobación
for i in range(5):
  Av = A_4@vectors[i]
  lambda_x = values[i]*vectors[i]
  print("Valor", i+1)
  print("Av = ", Av)
  print("lambda_x = ", lambda_x)
  print("----------------------------------------")

Valor 1
Av =  [2.1536204  3.72920167 4.30762758 3.73489439 2.15931804]
lambda_x =  [2.15445266 3.72850561 4.3060892  3.73558132 2.16154674]
----------------------------------------
Valor 2
Av =  [-2.99974216e+00 -2.99956629e+00  4.36584442e-04  3.00040083e+00
  3.00029064e+00]
lambda_x =  [-2.99981845e+00 -2.99957459e+00  2.83732468e-04  3.00031688e+00
  3.00027003e+00]
----------------------------------------
Valor 3
Av =  [ 2.30940025e+00  2.29356255e-07 -2.30939654e+00  8.08229651e-06
  2.30940644e+00]
lambda_x =  [ 2.30939969e+00 -1.46229889e-07 -2.30939894e+00  6.19604121e-06
  2.30940271e+00]
----------------------------------------
Valor 4
Av =  [-9.99991046e-01  1.00000398e+00 -1.79290897e-06 -9.99995059e-01
  1.00000992e+00]
lambda_x =  [-9.99996557e-01  1.00000159e+00 -2.17005271e-06 -9.99999044e-01
  1.00000472e+00]
----------------------------------------
Valor 5
Av =  [-0.15470045  0.26794924 -0.30940108  0.26794926 -0.15470041]
lambda_x =  [-0.15470055  0.26794924 -0.3094